In [43]:
import parser
import sparse_matrix
import predict

configs = {
    "k": 1.5,
    "b": 0.5,
    "alpha": 1,
    "beta": 0.75,
    "gamma": 0.15,
    "target": 100,
    "rochio": False,
    "query_path": "../queries/query-test.xml",
    "output_path": "../prediction.csv",
    "model_path": "../model",
    "corpus_path": "../CIRB010",

}

if __name__ == '__main__':
#     parser.parse_arg(configs)
    fname_to_id, id_to_fname, id_to_doclen = parser.parse_file_list(configs)
    vocab_to_id, id_to_vocab = parser.parse_vocab_list(configs)
    N = len(fname_to_id)
    inverted_files = parser.parse_inverted_file(configs, N)
    old_inverted_files = inverted_files.copy()
    

Reading Inverted Files: 


In [38]:
# Save checkpoint for notebook
inverted_files = old_inverted_files.copy()
avdl = sum(id_to_doclen.values()) / len(id_to_doclen)
corpus = {
    "fname_to_id": fname_to_id,
    "id_to_doclen": id_to_doclen,
    "id_to_fname": id_to_fname,
    "vocab_to_id": vocab_to_id,
    "id_to_vocab": id_to_vocab,
    "inverted_files": inverted_files,
    "avdl": avdl,
}
sparse_matrix.gen_matrix(corpus, configs)
id_to_magnitude = sparse_matrix.gen_id_to_magnitude(corpus, configs)
corpus["id_to_magnitude"] = id_to_magnitude

In [41]:
import parser
import sparse_matrix
import predict
import rocchio
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

configs["title_weight"] = 1
configs["question_weight"] = 1
configs["concepts_weight"] = 1
configs["narrative_weight"] = 1
configs["query_path"] = "../queries/query-test.xml"
configs["cdn"] = 1
configs["ctc"] = 1
configs["cte"] = 1
configs["cts"] = 1
configs["unigram_weight"] = 1
configs["bigram_weight"] = 1
configs["rocchio_iters"] = 1
configs["use_cosine"] = False

print("Processing Query")
queries = parser.parse_queries(corpus, configs, configs["query_path"])
queries = sparse_matrix.gen_query_vector(queries, corpus, configs)
query_to_magnitude = sparse_matrix.gen_query_to_magnitude(queries)
query_responses = []
for query in tqdm(queries):
    response = predict.predict_query(query, corpus, configs)
    query_responses.append(response)
#     print(response)

for _ in tqdm(range(configs["rocchio_iters"])):
    for i in tqdm(range(len(query_responses))):
        rocchio.rocchio_feedback(query_responses[i], queries[i],  corpus, configs)
        response = predict.predict_query(queries[i], corpus, configs)
        query_responses[i] = response


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing Query


In [42]:
predict.process_predictions(query_responses, configs)

print(query_responses[0])
predict.write_predictions(query_responses, queries)
predict.calc_MAP(query_responses, configs)

['cts_foc_0002188', 'cdn_foc_0002189', 'cdn_foc_0006865', 'cte_foc_0001449', 'cdn_foc_0006869', 'cdn_soc_0002239', 'cte_foc_0001454', 'cts_soc_0004302', 'cdn_foc_0006866', 'cts_soc_0004301', 'cts_foc_0002187', 'cdn_foc_0006868', 'cts_soc_0002407', 'cdn_foc_0006870', 'cts_foc_0002185', 'cts_esy_0000206', 'cts_soc_0005432', 'cts_soc_0005346', 'cte_foc_0001482', 'cts_foc_0002186', 'cts_soc_0002655', 'cdn_foc_0006872', 'cdn_soc_0002234', 'cdn_soc_0002235', 'cts_soc_0003399', 'cdn_foc_0006867', 'cts_foc_0002181', 'cts_soc_0002667', 'cts_for_0000848', 'cts_soc_0001344', 'cts_for_0000839', 'cts_soc_0001509', 'cts_foc_0001340', 'cte_foc_0002164', 'cte_foc_0001505', 'cts_foc_0003151', 'cts_soc_0003418', 'cts_foc_0001318', 'cts_foc_0003150', 'cte_foc_0002845', 'cts_soc_0002814', 'cts_soc_0004507', 'cts_chi_0001815', 'cts_soc_0001424', 'cts_foc_0005232', 'cts_soc_0005778', 'cts_soc_0004909', 'cte_foc_0003473', 'cts_foc_0002183', 'cts_soc_0002553', 'cts_soc_0001197', 'cts_soc_0004152', 'cdn_soc_00

### TODO!!!
* Add rochio
* Add logarithmic(?)
Add weighting to different components (title, 關鍵字)
Add weighting to different document source (cdn, cts, etc)